In [1]:
import json
import os
import pickle
from logging import INFO
from typing import Optional, Tuple, Union
import numpy as np

import pandas as pd
from analysis_backend.deep_learning_backend.deep_learning_backend import (
    DeepLearningBackend,
)
from analysis_backend.machine_learning_backend.machine_learning_backend import (
    MachineLearningBackend,
)
from flwr.client import NumPyClient
from flwr.common import Parameters
from flwr.common.logger import log
from src.flower_multifunction.task import set_model_parameters


class FlowerClientTrain(NumPyClient):
    def __init__(
        self,
        analysis_backend: Union[DeepLearningBackend, MachineLearningBackend],
        data: pd.DataFrame,
    ) -> None:
        self.analysis_backend = analysis_backend
        self.data = data

        self.temp_dir = os.path.join(os.getcwd(), "temp")
        self.model_path = os.path.join(self.temp_dir, "model.pkl")
        self.data_path = os.path.join(self.temp_dir, "data.pkl")
        self.config_path = os.path.join(self.temp_dir, "config.json")

    def prepare_data(
        self, analysis_config: dict
    ) -> tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]:

        data = self.analysis_backend.label_encoding(
            self.data,
            analysis_config["data_info"]["target_column"],
            encoding=analysis_config["data_info"]["encoding"],
        )

        indexes = self.analysis_backend.get_split_indexes(
            data,
            analysis_config["data_info"]["target_column"],
            split_strategy=analysis_config["train"]["split_strategie"]["name"],
            split_parameter=analysis_config["train"]["split_strategie"]["parameters"],
        )

        data_train = data.iloc[indexes[0]["train"]]
        # Extract features and targets from train and test sets
        X_train = data_train.drop(
            columns=[analysis_config["data_info"]["target_column"]]
        )
        y_train = data_train[analysis_config["data_info"]["target_column"]]

        data_test = data.iloc[indexes[0]["test"]]
        # Extract features and targets from train and test sets
        X_test = data_test.drop(columns=[analysis_config["data_info"]["target_column"]])
        y_test = data_test[analysis_config["data_info"]["target_column"]]

        return X_train, y_train, X_test, y_test

    def prepare_model(self, analysis_config: dict) -> object:

        model = self.analysis_backend.create_model(
            analysis_config["train"]["model"]["type"],
            analysis_config["train"]["model"]["parameters"],
        )
        log(INFO, f"{model.summary() = }")
        log(INFO, f"{self.analysis_backend = }")
        log(INFO, f"{analysis_config = }")
        log(INFO, f"{model.summary() = }")
        if isinstance(self.analysis_backend, DeepLearningBackend):
            log(INFO, "Using DeepLearningBackend to create the model.")
            model = self.analysis_backend.compile_model(
                model,
                analysis_config["train"]["model"]["compiler"],
            )
            log(INFO, f'{analysis_config["train"]["model"]["compiler"] = }')
        log(INFO, f"{model.summary() = }")

        return model

    def save_temp(
        self,
        model: object,
        X_train: pd.DataFrame,
        y_train: pd.Series,
        X_test: pd.DataFrame,
        y_test: pd.Series,
        analysis_config: dict,
    ) -> None:
        """Save model, data and config temporarily and delete old files"""
        os.makedirs(self.temp_dir, exist_ok=True)

        for path in [self.model_path, self.data_path, self.config_path]:
            if os.path.exists(path):
                os.remove(path)

        with open(self.model_path, "wb") as f:
            pickle.dump(model, f)

        with open(self.data_path, "wb") as f:
            pickle.dump((X_train, y_train, X_test, y_test), f)

        with open(self.config_path, "w") as f:
            json.dump(analysis_config, f)

    def load_temp(
        self,
    ) -> Tuple[
        Optional[object],
        Optional[pd.DataFrame],
        Optional[pd.Series],
        Optional[pd.DataFrame],
        Optional[pd.Series],
        Optional[dict],
    ]:
        """Load model, data and config"""
        if (
            os.path.exists(self.model_path)
            and os.path.exists(self.data_path)
            and os.path.exists(self.config_path)
        ):
            with open(self.model_path, "rb") as f:
                model = pickle.load(f)

            with open(self.data_path, "rb") as f:
                X_train, y_train, X_test, y_test = pickle.load(f)

            with open(self.config_path, "r") as f:
                analysis_config = json.load(f)

            return model, X_train, y_train, X_test, y_test, analysis_config
        return None, None, None, None, None, None

    def fit(self, parameters: Parameters, config: dict) -> tuple[float, int, dict]:

        # if config["current_round"] == 1:
        analysis_config = config # json.loads(config["config_json"])
        X_train, y_train, X_test, y_test = self.prepare_data(analysis_config)
        log(INFO, f"Before prepare_model - {analysis_config = }")
        model = self.prepare_model(analysis_config)
        log(INFO, f"After prepare_model - {analysis_config = }")

            # self.save_temp(model, X_train, y_train, X_test, y_test, analysis_config)

        # else:

        #     model, X_train, y_train, X_test, y_test, analysis_config = self.load_temp()

        model = set_model_parameters(model, parameters)

        log(
            INFO,
            f'{analysis_config["train"]["training"]["epochs"] = }',
        )
        log(
            INFO,
            f'{analysis_config["train"]["training"]["batch_size"] = }',
        )
        log(INFO, f"{self.analysis_backend = }")
        log(INFO, f"{model.summary() = }")

        model = self.analysis_backend.train_model(
            model,
            X_train,
            y_train,
            analysis_config["train"]["training"],
        )

        return model.get_weights(), len(X_train), {}

    def evaluate(self, parameters: Parameters, config: dict) -> tuple[float, int, dict]:

        log(INFO, "Evaluating model...")
        # log(INFO, f"{config['current_round'] = }")

        model, _, _, X_test, y_test, _ = self.load_temp()

        model = set_model_parameters(model, parameters)

        log(INFO, f"{self.analysis_backend = }")

        test_prediction = self.analysis_backend.predict(model, X_test)

        if len(test_prediction.shape) == 2:
            test_prediction = np.argmax(test_prediction, axis=1)
        test_validation = self.analysis_backend.validate(y_test, test_prediction)

        return 0, len(X_test), test_validation


2025-03-03 09:21:29.917063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-03 09:21:29.931592: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-03 09:21:29.935870: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 09:21:29.946335: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-03 09:21:30.784456: W tensorflow/compiler/tf2

In [2]:
from src.flower_multifunction.task import load_data, load_config

analysis_backend = DeepLearningBackend()
data = load_data(0, 2)
analysis_config = load_config()

# Return Client instance
test = FlowerClientTrain(analysis_backend, data)

In [3]:
prepare_model = test.prepare_model(analysis_config)

I0000 00:00:1740990094.847555   16243 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1740990094.867379   16243 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1740990094.868455   16243 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1740990094.871742   16243 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1740990094.872602   16243 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 193 (772.00 B)

 Trainable params: 193 (772.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 193 (772.00 B)

 Trainable params: 193 (772.00 B)

 Non-trainable params: 0 (0.00 B)

INFO :      model.summary() = None


In [4]:
model = set_model_parameters(prepare_model, prepare_model.get_weights())
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 193 (772.00 B)

 Trainable params: 193 (772.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
# data = test.analysis_backend.label_encoding(
#     test.data,
#     analysis_config["data_info"]["target_column"],
#     encoding=analysis_config["data_info"]["encoding"],
# )

# y = data[["Iris-setosa", "Iris-versicolor", "Iris-virginica"]]
# x = data.drop(columns=["Iris-setosa", "Iris-versicolor", "Iris-virginica"])

In [6]:
X_train, y_train, X_test, y_test = test.prepare_data(analysis_config)

In [7]:
X_test

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
18,7.2,3.2,6.0,1.8
20,4.9,3.1,1.5,0.1
13,6.4,2.7,5.3,1.9
16,4.7,3.2,1.3,0.2
1,5.7,2.6,3.5,1.0
51,5.8,2.7,5.1,1.9
43,7.7,3.0,6.1,2.3
24,5.1,3.5,1.4,0.2
58,6.4,3.2,4.5,1.5
50,7.0,3.2,4.7,1.4


In [8]:
test.fit(prepare_model.get_weights(), analysis_config)

INFO :      Before prepare_model - analysis_config = {'data_info': {'encoding': 'integer', 'target_column': 'Species'}, 'train': {'split_strategie': {'name': 'train_test_split', 'parameters': {'test_size': 0.2, 'shuffle': True}}, 'model': {'type': 'mlp', 'parameters': [{'type': 'Input', 'shape': [4]}, {'type': 'Dense', 'units': 10, 'activation': 'relu'}, {'type': 'Dense', 'units': 10, 'activation': 'relu'}, {'type': 'Dense', 'units': 3, 'activation': 'softmax'}], 'compiler': {'optimizer': {'class_name': 'adam', 'config': {'learning_rate': 0.001}}, 'loss': 'sparse_categorical_crossentropy', 'metrics': ['accuracy']}}, 'training': {'epochs': 5, 'batch_size': 2, 'validation_split': 0.2, 'verbose': 0}}, 'statistic': {'mean': {'dataframe_parameters': {'columns': 'all'}, 'statistic_parameters': {'axis': 0}}, 'median': {'dataframe_parameters': {'columns': 'all'}, 'statistic_parameters': {'axis': 0}}, 'std': {'dataframe_parameters': {'columns': 'all'}, 'statistic_parameters': {'axis': 0}}, 'var

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 10)             │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 193 (772.00 B)

 Trainable params: 193 (772.00 B)

...

 Total params: 193 (772.00 B)

 Trainable params: 193 (772.00 B)

 Non-trainable params: 0 (0.00 B)

INFO :      model.summary() = None
I0000 00:00:1740990097.829121   16351 service.cc:146] XLA service 0x7f4298004040 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740990097.829160   16351 service.cc:154]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2025-03-03 09:21:37.845577: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-03-03 09:21:39.647248: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
I0000 00:00:1740990100.217801   16351 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


([array([[-0.15231697,  0.3885914 ,  0.51305693, -0.13524657,  0.05054283,
          -0.38550267,  0.33695373, -0.5881935 , -0.46319976, -0.5295861 ],
         [ 0.5431633 , -0.58527637, -0.06835156,  0.09628129, -0.16019309,
           0.22655696,  0.54281855, -0.30138484, -0.37610608,  0.29466397],
         [ 0.53857833, -0.4434982 ,  0.47383067, -0.14515197, -0.14027834,
          -0.29616374, -0.42466217, -0.63863266,  0.4059664 ,  0.34394938],
         [-0.02797021, -0.2584399 ,  0.35960817, -0.04747158, -0.37308386,
          -0.2490296 , -0.05046504, -0.11403102, -0.37486288,  0.04700333]],
        dtype=float32),
  array([-0.05842457,  0.        , -0.03567658,  0.        ,  0.        ,
          0.        ,  0.028619  ,  0.        ,  0.        ,  0.        ],
        dtype=float32),
  array([[-0.46720898, -0.25369325, -0.3968445 ,  0.13315122, -0.00771537,
           0.06903751, -0.02121884,  0.2392238 ,  0.5097785 ,  0.20802766],
         [ 0.22603434,  0.38539338, -0.13525352

In [9]:
test.evaluate(prepare_model.get_weights(), analysis_config)

INFO :      Evaluating model...
/home/dk100/miniconda3/envs/flower_test/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
INFO :      self.analysis_backend = <analysis_backend.deep_learning_backend.deep_learning_backend.DeepLearningBackend object at 0x7f43df03cda0>


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step


(0, 10, {'accuracy': 0.3})

In [10]:
model, _, _, X_test, y_test, _ = test.load_temp()

/home/dk100/miniconda3/envs/flower_test/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [11]:
predict = model.predict(x)

NameError: name 'x' is not defined

In [25]:
y_onehot = label_encoding

NameError: name 'label_encoding' is not defined

In [28]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

accuracy_score(np.argmax(predict, axis=1),np.argmax(y, axis=1))

0.38666666666666666